<a href="https://colab.research.google.com/github/emerenan/xlsx_validation/blob/main/ie_validations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist


Towerdb File

In [27]:
""" Need Unlock spreadsheet before"""
path='/content/TowerDB_Ireland_20210630 v1.xlsx'
sheet='TowerDB_Input_Ireland_20210630'
skiprows=0
skipcols=0
"""A coluna Infrastructure Ready Date contem 0 e deve ser feito replace"""
towerdb = pd.read_excel(path, sheet_name = sheet).fillna('')
towerdb = towerdb.dropna(subset=['Site ID '], axis=0)

dates_tw = ['Infrastructure Ready Date','Billing Trigger Date', 'BTS Site Acceptance Date', 'First_Active_Sharing_Start_Date', 'First_Active_Sharing_End_Date']
for col in dates_tw:
    lista = []
    towerdb[col] = towerdb[col].replace([0, '#N/D', 'N/A', np.nan, 'nan', '-'], '')
    for dt in towerdb[col]:
        if dt != '' or len(dt) > 0:
            new_dt = f"{dt:%d/%m/%Y}"
            lista.append(new_dt)
        else:
            lista.append(dt)
    towerdb[col] = lista

#Read CZ towerdb files
"""Merge to get DOER column"""
msa = pd.read_csv('/content/TowerDB_Ireland_20210731.csv', engine='python')
doer_col = msa[['Site ID ', 'Date of Equipment Removal']]
towerdb = towerdb.merge(doer_col, on='Site ID ')

lista = []
towerdb['Date of Equipment Removal'] = towerdb['Date of Equipment Removal'].replace([0, '#N/D', 'N/A', np.nan, 'nan'],'')
for dt in towerdb['Date of Equipment Removal']:
    if dt != '' and re.match(r"[0-9]{1,2}[\_|\-|\/][0-9]{1,2}[\_|\-|\/][0-9]{1,4}", dt) is None:
        new_dt = f"{dt:%d/%m/%Y}"
        lista.append(new_dt)
    else:
        lista.append(dt)
towerdb['Date of Equipment Removal'] = lista

towerdb.rename(columns={'Billing Trigger Date': 'Billing Trigger Date.1'}, inplace=True)
towerdb['Billing Trigger Date'] = towerdb['Infrastructure Ready Date'].values

towerdb["Energy Cost FY1819"] = towerdb["Energy Cost FY1819"].apply(lambda x: '€ {:0,.2f}'.format(x))
#towerdb['First_Active_Sharing_Start_Date'] = towerdb['First_Active_Sharing_Start_Date'].replace('-', '')
#towerdb['First_Active_Sharing_End_Date'] = towerdb['First_Active_Sharing_End_Date'].replace('-', '')

cols = list(msa.columns)
towerdb = towerdb[cols]

replace_cols = ['Site ID ','Strategic','Critical', 'Site Type (See structure Type)','Core Site Type','Transmission System',\
             'Macro Site - Transmission Hub Site','Macro Site - Transmission Hub Site with/without Shelters','Transmission Sites', 'Room Configuration',\
             'Power Supply','Air Conditioning','Strategic_Site_Bucket','CriticalSite_Beyond_10','Sites_As_Metered_Estimated','Meter_Sharing_Site','Bts_Sites',\
             'Phase1_Site_Or_Consent_Site','WIP_Sites','Active_Sharing_Arrangement','Subsequent_Sharing_Arrangement','First_Active_Sharing_Deployment_Type',\
             'Billing Trigger Date','Date of Equipment Removal']

invalid_values = ['N/A', 'n/a',"0" ,0, '_', np.nan,'nan', ' ']
for col in replace_cols:
    towerdb[col] = towerdb[col].replace(invalid_values, '')

towerdb.rename(columns={'Billing Trigger Date.1': 'Billing Trigger Date'}, inplace=True)

towerdb.head(3)

,Site ID,Population,County,Site Name,Address,Site Provider,Strategic,Critical,Altitude (ASL),Lat (WGS 84),Long (WGS 84),Site Class,Site Owner,Hub List,Site Type (See structure Type),Active Dishes,INTP,Core Site Type,Transmission System,Macro Site - Transmission Hub Site,Macro Site - Transmission Hub Site with/without Shelters,Transmission Sites,Public DAS Sites (Active/ Passive),Fibre/Microwave,Technology VOD,Structure Type,Structure Type (Standardised),Room Configuration,Power Supply,Air Conditioning,Tower Height,Site Area (sqm),Site Area (Available),Macro Region,MPRN,Site Decom,SAP Site Code,Energy Cost FY1819,Active / Not Active,New Build planned,Site Status,Planned Sublets,Infrastructure Ready Date,Strategic_Site_Bucket,CriticalSite_Beyond_10,Sites_As_Metered_Estimated,Meter_Sharing_Site,Bts_Sites,Phase1_Site_Or_Consent_Site,Transfer_Date_Of_Consent_Site,Billing Trigger Date,WIP_Sites,Decommisioned_Site,Active_Sharing_Arrangement,Subsequent_Sharing_Arrangement,First_Active_Sharing_Deployment_Type,First_Active_Sharing_Start_Date,First_Active_Sharing_End_Date,BTS Site Acceptance Date,Billing Trigger Date,Date of Equipment Removal
0,CE009,183,Clare,Dangan,"Dangan, Ballyvaughan, Co Care",Private Individual,No,No,47,53.1011,-9.12983,Rural,Vodafone,P4,GBT,2,Standard,Non Core,Macro,No,Non Transmission Hub Site,Non Transmission Site,0,Microwave,2G; 3G,Lattice,Lattice,Indoor,AC,No,18,400,80,Mid West,1.00097e+10,In Service,IECE009,"€ 1,229.31",Active,,In Service,,01/08/1998,Non Strategic,Non Critical,Estimated Model,No,No,MSA (Phase 1),,01/08/1998,No,No,No Active Sharing,No,No,,,,,
1,CE015,1229,Clare,Bunratty Castle Hotel,Bunratty Castle Hotel Bunratty Clare,Hotel Company,No,No,12,52.6962,-8.81526,Rural,Private Company,P4,RTT,1,Standard,Non Core,Macro,No,Non Transmission Hub Site,Non Transmission Site,0,Microwave,2G; 3G; 4G,Rooftop structure,Rooftop structure,KN type Outdoor,AC,No,13.4,,,Mid West,0,In Service,IECE015,"€ 1,853.73",Active,,In Service,,07/05/2007,Non Strategic,Non Critical,Estimated Model,No,No,MSA (Phase 1),,07/05/2007,No,No,No Active Sharing,No,No,,,,,
2,CE020,1143,Clare,Liscannor Bay Hotel,Liscannor Bay View Hotel Liscannor Clare,Hotel Company,No,No,11,52.9382,-9.38929,Rural,Private Company,P4,RTT,1,Standard,Non Core,Macro,No,Non Transmission Hub Site,Non Transmission Site,0,Microwave,2G; 3G,Rooftop structure,Rooftop structure,KN type Outdoor,AC,No,19.2,,,Mid West,1.0019e+10,In Service,IECE020,€ 995.32,Active,,In Service,,21/05/2002,Non Strategic,Non Critical,Estimated Model,No,No,MSA (Phase 1),,21/05/2002,No,No,No Active Sharing,No,No,,,,,


In [ ]:
def change_incorret_values(df, df_index, col_chg, incorrect_value, new_value):
    list_sites = list(df[df[col_chg]==incorrect_value][df_index])
    df.loc[df[df_index].isin(list_sites), col_chg] = new_value
    print(df.loc[df[df_index].isin(list_sites)][[df_index, col_chg]])

def change_incorret_list(df,list_sites, df_index, col_chg, new_value):
    #list_sites = list(df[df[col_chg]==incorrect_value][df_index])
    df.loc[df[df_index].isin(list_sites), col_chg] = new_value
    print(df.loc[df[df_index].isin(list_sites)][[df_index, col_chg]])

lista = ['CK166', 'CK173', 'CKMTG','DN370','DNTS1','DNVIC','GY135','KK074','RN053','RN058','SO071','WD067']
change_incorret_list(towerdb, lista, 'Site ID ', "Site Status", 'To be dismantled')

b_sites = ['CK094','DX264','GY025','DN947']
change_incorret_list(towerdb, b_sites,'Site ID ',"Bts_Sites", 'Yes')


In [ ]:
"""Precisa dos Critical sites da UIS"""
c_sites = critical['UIS Sites(critical beyond 10%) different in TowerDB'].to_list()
change_incorret_list(towerdb, c_sites,'Site ID ','CriticalSite_Beyond_10', 'Beyond 10%')

In [ ]:
towerdb.to_csv('/content/TowerDB_Ireland_20210831.csv', index=False)

In [ ]:
"""Check columns received"""
def check_columns_received(df, bill_cols):
    twdb_col = list(df.columns)
    col_miss = [i for i in bill_cols if i not in twdb_col]

    if col_miss:
        df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'])
        return df_col_missing
    else:
        print('\nNo missing columns.')

df_col_missing = check_columns_received(towerdb, cols)
df_col_missing
#Tem error 3 columns faltando

,Column(s) Missing
0,Billing Trigger Date.1


In [ ]:
def check_date_columns(df, df_index,status_col,columns):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    from collections import defaultdict

    df_dates = df[columns].fillna('')
    df_dates = df_dates.to_dict(orient='list')

    new_dic = defaultdict(list)

    date_format = re.compile(r"[0-9]{1,2}[\_|\-|\/][0-9]{1,2}[\_|\-|\/][0-9]{1,4}")

    for column in set(df_dates.keys()):
        for value in df_dates[column]:
            if date_format.match(value) is None:
                if value in (None, '', 'nan'):
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorret picklist value: {value}')
            else:
                new_dic[column].append('Ok')

    df_errors = pd.DataFrame(new_dic)
    df_errors[df_index] = [i for i in df[df_index]]
    df_errors = df_errors.replace('Ok', np.nan)   
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()
    
    df = df[[df_index, status_col]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[status_col]+ df_errors.columns[:-1].tolist()]
    df_errors = df_errors.reset_index()
    return df_errors

towerdb = towerdb.T.drop_duplicates().T

actives_1 = towerdb[towerdb['Site Status']=='In Service']
no_actives_1 = towerdb[~(towerdb['Site Status']=='In Service')]

#Checking columns for errors
actives_dates_errors = check_date_columns(actives_1, 'Site ID ', 'Site Status', ['Billing Trigger Date']) 
# Actives sites with blank billing trigger date
no_actives_dates_errors = check_date_columns(no_actives_1, 'Site ID ', 'Site Status', ['Date of Equipment Removal']) 

print(actives_dates_errors)
print('\n')
print(no_actives_dates_errors)

In [32]:
def check_picklist(df, df_index, df_status, picklist_dict):
    from collections import defaultdict

    df_cols = list(picklist_dict.keys())
    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)

    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]):
                if pd.isnull(value) or pd.isna(value) or value == '' or value == 'nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors[df_index] = [i for i in df[df_index]]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, df_status]]
    df_errors = pd.merge(df_errors, df, how='left', on=[df_index], suffixes=('_checked', '_actual'))
    cols = list(df_errors.columns)
    if df_status + '_actual' in cols:
        df_errors = df_errors.set_index([df_index, df_status + '_actual'])
    else:
        df_errors = df_errors.set_index([df_index])
        df_errors = df_errors[[df_status] + df_errors.columns[:-1].tolist()]
    df_errors = df_errors.reset_index()
    return df_errors

general_picklist = {
    'Transmission System': ["Macro","Public DAS","Long-term Mobile","Transmission",'Repeater','Macro+DAS','Outdoor Small Cells'],
}

df_general_picklist = check_picklist(towerdb, 'Site ID ','Site Status', general_picklist)

print(df_general_picklist)


  Site ID  Site Status Transmission System
0    DN133  Dismantled         Blank Value
1    DN214  Dismantled         Blank Value
2    DN359  Dismantled         Blank Value
3    DNSE1  Dismantled         Blank Value
4    WX035  Dismantled         Blank Value
5    LK568    Pipeline         Blank Value
6    MH079    Pipeline         Blank Value


In [ ]:
"""MoM BTS check sites"""
def check_mom_bts(df_tw,df_msa, df_index, status_col, check_col):

    msa_bts = df_msa[df_msa[check_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[df_index]]

    tw_bts = df_tw[df_tw[check_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[df_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = df_msa[df_msa[df_index].isin(out_tower_bts)]
    filtered = filtered[[df_index, status_col, check_col]]
    actual = df_tw[df_tw[df_index].isin(out_tower_bts)][[df_index, check_col, status_col]]
    merge = pd.merge(filtered, actual, how='inner', on=df_index, suffixes=['_old', '_in_month'])

    return  merge[[df_index, status_col+'_old', status_col+'_in_month', check_col+'_old', check_col+'_in_month']]

df_mom = check_mom_bts(towerdb, msa , 'Site ID ','Site Status', "Bts_Sites")
df_mom

,Site ID,Site Status_old,Site Status_in_month,Bts_Sites_old,Bts_Sites_in_month
0,CK094,In Service,In Service,Yes,No
1,DN947,In Service,In Service,Yes,No
2,DX264,In Service,In Service,Yes,No
3,GY025,In Service,In Service,Yes,No


In [ ]:
"""MoM DEcomissioned check sites"""
def check_mom_bts(df_tw,df_msa, df_index, status_col, check_col):

    msa_bts = df_msa[df_msa[check_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[df_index]]

    tw_bts = df_tw[df_tw[check_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[df_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = df_msa[df_msa[df_index].isin(out_tower_bts)]
    filtered = filtered[[df_index, status_col, check_col]]
    actual = df_tw[df_tw[df_index].isin(out_tower_bts)][[df_index, check_col, status_col]]
    merge = pd.merge(filtered, actual, how='inner', on=df_index, suffixes=['_old', '_in_month'])

    return  merge[[df_index, status_col+'_old', status_col+'_in_month', check_col+'_old', check_col+'_in_month']]

df_decom = check_mom_bts(towerdb, msa, 'Site ID ', 'Site Status', 'Decommisioned_Site')


,Site ID,Site Status_old,Site Status_in_month,Decommisioned_Site_old,Decommisioned_Site_in_month


In [30]:
def check_picklist(df, df_index, df_status, picklist_dict):
    from collections import defaultdict

    df_cols = list(picklist_dict.keys())
    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)

    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]):
                if pd.isnull(value) or pd.isna(value) or value == '' or value == 'nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors[df_index] = [i for i in df[df_index]]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, df_status]]
    df_errors = pd.merge(df_errors, df, how='left', on=[df_index], suffixes=('_checked', '_actual'))
    cols = list(df_errors.columns)
    if df_status + '_actual' in cols:
        df_errors = df_errors.set_index([df_index, df_status + '_actual'])
    else:
        df_errors = df_errors.set_index([df_index])
        df_errors = df_errors[[df_status] + df_errors.columns[:-1].tolist()]
    df_errors = df_errors.reset_index()
    return df_errors


picklist = {
    "Strategic": ['Yes', 'No'],
    'Critical': ['Yes', 'No'],
    'Site Type (See structure Type)': ['RTT','GBT','DAS active','DAS passive','RTT+DAS active','GBT+DAS active','RTT+DAS passive','GBT+DAS passive','Outdoor Small Cells'],
    'INTP ': ['Standard','Core','Dual Use Site','Transmission','Repeater','Long-term Mobile','DAS active','DAS passive','Macro+DAS active','Macro+DAS passive','Outdoor Small Cells'],
    'Core Site Type': ['Non Core', 'Case A', 'Case B'],
    'Transmission System': ["Macro","Public DAS","Long-term Mobile","Transmission",'Repeater','Macro+DAS','Outdoor Small Cells'],
    'Macro Site - Transmission Hub Site': ['Yes', 'No'],
    'Macro Site - Transmission Hub Site with/without Shelters': ["With shelters","Without shelters","Non Transmission Hub Site"],
    'Transmission Sites': ["With shelters","Without shelters","Non Transmission Site"],
    'Room Configuration': ['Indoor','Outdoor','Indoor & Outdoor', "KN type Outdoor","RBS type Outdoor"],
    'Power Supply': ['AC','DC', 'AC & DC','No Power'],
    'Air Conditioning': ['No','Yes; Indoor Air Conditioning','Yes; Indoor Free Air cooling / Free cooling units'],
    'Site Status': ['Pipeline','Ordered','WIP','In Service','To be dismantled','Dismantled','Cancelled'],
    'Strategic_Site_Bucket': ['Yes - 0-5%','Yes - 5-10%','Non Strategic'],
    'CriticalSite_Beyond_10': ['Beyond 10%','Within 10%','Non Critical'],
    'Sites_As_Metered_Estimated': ['Estimated Model','Metered Model'],
    'Meter_Sharing_Site': ['Yes', 'No'],
    "Bts_Sites": ['Yes', 'No'],
    'Phase1_Site_Or_Consent_Site': ['MSA (Phase 1)','PMA (Phase 2)'],
    'WIP_Sites': ['Yes', 'No'],
    'Active_Sharing_Arrangement': ['MORAN (On VF equipment)','MORAN (On non-VF equipment)','MOCN with Spectrum Pooling','Partial Active-Passive','No Active Sharing'],
    'Subsequent_Sharing_Arrangement': ['Yes', 'No']

}
"""Picklist check"""
actives = towerdb[towerdb['Site Status']=='In Service']

"""Picklist check"""
actives = towerdb[towerdb['Site Status']=='In Service']
df_active_picklist = check_picklist(actives, 'Site ID ','Site Status',  picklist)

print(df_active_picklist)

  Site ID  Site Status Subsequent_Sharing_Arrangement
0    WH032  In Service                    Blank Value
1    DN304  In Service                    Blank Value
2    DN947  In Service                    Blank Value
3    DX252  In Service                    Blank Value


In [ ]:
path_uip = '/content/UserInput_Ireland_July 21_06.07.2021.xlsx'
uip_names = ['Site_ID', 'BTS site applicable charge (Annual)', \
             'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip = pd.read_excel(path_uip ,sheet_name='SiteLevel',usecols=[0,1,2],skiprows=2).fillna('')
uip.columns = uip_names

msa_sites = [i for i in msa['Site ID ']]
tw_sites = [i for i in towerdb ['Site ID ']]
uip_sites = [i for i in uip['Site_ID']]


In [ ]:
"""Check Bill in service sites"""
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
        
    if t == 'doer':
        df_tw[date_col]= pd.to_datetime(df_tw[date_col], format='%d/%m/%Y', errors='coerce')
        filtered = df_tw[(df_tw[status_col]==status)&((df_tw[date_col] < current_date)|(df_tw[date_col]==''))]
        filtered[date_col] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[date_col]))
        return filtered[[tw_index, status_col, date_col]] 
    else:

        filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col]=='')]
        return filtered[[tw_index, status_col, date_col]] 

#check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col)       
df_bill = check_tw_bill_doer(actives, 'Site ID ', 'Billing Trigger Date',"Bts_Sites",'Yes', 'bill')
df_bill

In [ ]:
"""BTS Wip"""
def check_wip(df_tw,  df_msa, tw_index, wip_tw, tw_bts, tw_status):

    wip_msa = [i for i in df_msa[df_msa[wip_tw]=='Yes'][tw_index]]
    
    tw_wip_sites = [str(i) for i in df_tw[df_tw[wip_tw]=='Yes'][tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[wip_tw]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index, wip_tw, tw_bts]]
    
    #wip_out_tw_list = [i for i in tw_wip_sites if i not in wip_msa]
    return tw_wip_site_bts_flagged

df_wip_and_bts_flagged = check_wip(towerdb, msa, 'Site ID ','WIP_Sites', "Bts_Sites","Site Status")

print(df_wip_and_bts_flagged)
# No errors

In [ ]:
"""Decomissioned Sites"""
def check_decommissioned(df,df_index, decom_col, doer_col):
    #c = country.lower()
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    return filtered[[df_index, decom_col, doer_col]]
actives = towerdb[towerdb['Site Status']=='in Service']
df_decom_errors = check_decommissioned(actives,'Site ID ','Decommisioned_Site', 'Date of Equipment Removal')
df_decom_errors
# No errors


,Site ID,Decommisioned_Site,Date of Equipment Removal


In [ ]:
""" BTS sites"""
def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    filtered = df_towerdb[[tw_index, status_col]]

    #Finding for ALL NEW SITES
    new_site = [i for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    new_sites = pd.merge(new_sites, filtered, how='left', left_on=['New_Sites'], right_on=tw_index)
    new_sites = new_sites[['New_Sites', status_col]]

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = pd.to_datetime(df[bill_col], format='%d/%m/%Y', errors='coerce')
    df_site_bts = df[(df[bts_col]=='Yes')&(df[bill_col] > current_date)].fillna('')
    df_site_bts[bill_col] = list(map(lambda x: f'{x:%d/%m/%Y}', df_site_bts[bill_col]))
    df_site_bts = df_site_bts[[tw_index,status_col, bts_col, bill_col]]
    
    #if not (new_sites.empty or bts_out_uis.empty or df_site_bts.empty):
    return new_sites, df_site_bts[[tw_index,status_col, bts_col, bill_col]]

#(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list, uip_list)
new_sites, bts_out_uis = check_new_sites(towerdb, 'Site ID ', "Bts_Sites", \
                                                      'Billing Trigger Date','Site Status', msa_sites, tw_sites, uip_sites)

print(new_sites)
print('\n')
print(bts_out_uis)
#no errors

In [ ]:
"""Check doer in service sites"""
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
        
    if t == 'doer':
        df_tw[date_col]= pd.to_datetime(df_tw[date_col], format='%d/%m/%Y', errors='coerce')
        filtered = df_tw[(df_tw[status_col]==status)&((df_tw[date_col] < current_date)|(df_tw[date_col]==''))]
        filtered[date_col] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[date_col]))
        return filtered[[tw_index, status_col, date_col]] 
    else:

        filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col]=='')]
        return filtered[[tw_index, status_col, date_col]] 

#check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col)       
df_doer = check_tw_bill_doer(actives, 'Site ID ', 'Date of Equipment Removal','Site Status','In Service', 'doer')
df_doer
# No errors


,Site ID,Site Status,Date of Equipment Removal


In [ ]:
def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col, tw_critical_col]]
    #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]
    uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS In Month not active in TowerDB!'])
    uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS In Month not active in TowerDB!',\
                                    right_on=tw_index)
    uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS In Month not active in TowerDB!', status_tw_col]]
    
    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS In Month!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS In Month!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS In Month!', status_tw_col]]
    #check for decomissioned site not in uip files

    tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
    decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
    decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                    right_on=tw_index)
    decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]

    #Check BTS sites
    bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
    uip_bts = [i for i in df_uip[df_uip['BTS site applicable charge (Annual)']!='']['Site_ID']]

    #if not set(bts_sites).intersection(uip_sites):
    bts_sites_out_uip = [i for i in uip_bts if i not in bts_sites]
    bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['UIS BTS not in TowerDB(BTS)'])
    bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='UIS BTS not in TowerDB(BTS)',\
                                    right_on=tw_index)
    bts_sites_out_uip = bts_sites_out_uip[['UIS BTS not in TowerDB(BTS)', status_tw_col]]

    #  Check for UIP critical sites 
    uip_critical = [i for i in df_uip[df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='']['Site_ID']]
    bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
    
    #if set(uip_critical).intersection(bts_tw_critical):
    critical = [i for i in uip_critical if i not in bts_tw_critical]
    critical = pd.DataFrame(critical, columns=['UIS Sites(critical beyond 10%) different in TowerDB'])
    critical = pd.merge(critical, filtered, how='left', left_on='UIS Sites(critical beyond 10%) different in TowerDB',\
                                right_on=tw_index)
    critical = critical[['UIS Sites(critical beyond 10%) different in TowerDB', status_tw_col, tw_critical_col]]
    #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
    return uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical

"""Check UIP sites matches with Towerdb sites"""
uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical = check_uip_tw(towerdb,'Site ID ', 'Site Status', \
                                                              'Decommisioned_Site', "Bts_Sites",'CriticalSite_Beyond_10', uip, uip_sites)
print(uis_sites_not_in_towerdb)
print('\n')
print(in_service_not_in_uis)
print('\n')
print(decomiss_sites_in_uip)
print('\n')
print(bts_sites_out_uip)
print('\n')
print(critical)

True up check

In [ ]:
def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col, tw_critical_col]]
    #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]
    uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS In Month not active in TowerDB!'])
    uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS In Month not active in TowerDB!',\
                                    right_on=tw_index)
    uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS In Month not active in TowerDB!', status_tw_col]]
    
    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS In Month!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS In Month!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS In Month!', status_tw_col]]
    #check for decomissioned site not in uip files

    tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
    decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
    decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                    right_on=tw_index)
    decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]

    #Check BTS sites
    bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
    uip_bts = [i for i in df_uip[df_uip['BTS site applicable charge (Annual)']!='']['Site_ID']]

    #if not set(bts_sites).intersection(uip_sites):
    bts_sites_out_uip = [i for i in uip_bts if i not in bts_sites]
    bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['UIS BTS not in TowerDB(BTS)'])
    bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='UIS BTS not in TowerDB(BTS)',\
                                    right_on=tw_index)
    bts_sites_out_uip = bts_sites_out_uip[['UIS BTS not in TowerDB(BTS)', status_tw_col]]

    #  Check for UIP critical sites 
    uip_critical = [i for i in df_uip[df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='']['Site_ID']]
    bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
    
    #if set(uip_critical).intersection(bts_tw_critical):
    critical = [i for i in uip_critical if i not in bts_tw_critical]
    critical = pd.DataFrame(critical, columns=['UIS Sites(critical beyond 10%) different in TowerDB'])
    critical = pd.merge(critical, filtered, how='left', left_on='UIS Sites(critical beyond 10%) different in TowerDB',\
                                right_on=tw_index)
    critical = critical[['UIS Sites(critical beyond 10%) different in TowerDB', status_tw_col, tw_critical_col]]
    #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
    return uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical

path_uiptrue = '/content/UserInput_Ireland_20210630 June delta.xlsx'
uip_names = ['Site_ID', 'BTS site applicable charge (Annual)', 'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip_true = pd.read_excel(path_uip ,sheet_name='SiteLevel',usecols=[0,1,2],skiprows=2)
uip_true.columns = uip_names
uip_sites_true = [i for i in uip_true['Site_ID']]

uis_true_not_in_towerdb,in_service_uis_true, decomiss_in_uis, bts_sites_out_uis_true, critical_true = check_uip_tw(towerdb,'Site ID ', 'Site Status', \
                                                                'Decommisioned_Site', "Bts_Sites",'CriticalSite_Beyond_10', uip, uip_sites)
#No errors
print(uis_sites_not_in_towerdb)
print('\n')
print(in_service_not_in_uis)
print('\n')
print(decomiss_sites_in_uip)
print('\n')
print(bts_sites_out_uip)
print('\n')
print(critical)

In [ ]:
def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan']
        #lista = []

        df.fillna("", inplace=True)
        #df[column] = df[column].astype('int64')

        for column in columns:
            lista = []
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista

        return df

    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names)
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names )
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols)

    #df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    if not df_commercial_diffs.empty:
        return df_commercial_diffs
    else:
        print('\nNo Errors founded!')

names = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', 'Input_Value',\
         'Description/Instruction', 'Frequency of Update']
merge_cols = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', \
              'Description/Instruction', 'Frequency of Update']
cols_ordered = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2','Data_Type','Input_Value_actual',\
                'Input_Value_before','Equal Values','Description/Instruction', 'Frequency of Update']
# Check for commercial Values into current UIP File and compare with UIP File before
path_before = '/content/UserInput_Ireland_20210731.xlsx'
path_curr = '/content/UserInput_Ireland_July 21_06.07.2021.xlsx'
df_com = check_commercial(path_curr, path_before,['Input_Value'], names, merge_cols, cols_ordered)
df_com

Excel log

In [ ]:
def general_log_erros(df_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')  
    for lista in df_list:
        try:
            if not lista[1].empty:
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
        except AttributeError:
            pass
        except:
            if lista[1] is not None and lista[1].is_valid():
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
    try: 
        writer.save() 
    except IndexError:
        print('No one log to write!')

df_tw = [['In Service Date Errors',actives_dates_errors],
        ['Dismantled Dates Errors',no_actives_dates_errors],
        ['General Picklist Errors',df_general_picklist],
        ['Decom Status Error',df_decom],
        ['In Service Picklist Errors',df_active_picklist],
        ['Billing Dates Errors',df_bill],
        ['WIP & BTS Flagged',df_wip_and_bts_flagged],
        ['Decom dates Errors',df_decom_errors],
        ['New Sites',new_sites],
        ['New Sites Bill date errors',bts_out_uis],
        ['DOER Dates Errors',df_doer],
        ['UIS In Month not active', uis_sites_not_in_towerdb],
        ['TowerDB Sites out of UIS', in_service_not_in_uis],
        ['Decom Sites in UIS', decomiss_sites_in_uip],
        ['Critical Sites out UIS', critical],
        ['Commercial Differences' , df_com]] 

path_tw = '/content/towerdb_IE_errors.xlsx'
general_log_erros(df_tw, path_tw)

TA Input File

In [ ]:
#Read TA Input File
def read_files(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int,\
               cols_amount, bill_cols, format='mix', type_date="%d/%m/%Y"):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df
    def date_parser(df, columns,format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)
    #header=0, names = col_name,
    df = pd.read_excel(path, sheet_name = sheetname, engine='openpyxl', skiprows = n_skiprows)
    df = df.iloc[:,n_skip_columns:]
    df = df[col_name] 
    df = df.dropna(subset=[site_index], axis=0)
    df = replace_values(df, cols_amount, 0)
    for col in cols_amount:
        df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int).apply(lambda x: f'{x:,}')
    df = replace_values(df, cols_int, 0)
    for col in cols_int:
        df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int)
    df = df.fillna('')
    df = replace_values(df, bill_cols)
    date_parser(df, cols_date, format, type_date)
    # define the entiry columns which doesn't have NaN 
    return df
ta_col_ord = ["Site ID ","Site Name ","Address ","Site Provider","Site type",\
              "Tenant/Customer ","Customer Type "," Initial Rent  "," Current Rent  "," REC Uplift Charge ",\
              "Current Power Charge","Renegotiations ","Lease Start Date ","Lease End date ","Right to Renew ",\
              "End of Renewal Period ","Payment Terms ","Payment Frequency ","Rent Review ","Indexation Driver",\
              "Fixed Increase ","CPI ","Percentage","Increase frequency","Increase Date","VAT","Rate ","Termination date "]
interger = ['Increase frequency']
amount = []
ta_bill_cols = ["Site ID ","Tenant/Customer ","Customer Type ",\
                "Lease Start Date ","Lease End date "]
ta_dates = ["Lease Start Date ","Lease End date ", "Increase Date", 'Termination date ']
              
path='/content/TowerDB_Ireland_20210630 v1.xlsx'
sheet='TA_Input_Ireland_20210630'
skiprows=0
skipcols=0
#(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int,\
               #cols_amount, bill_cols, format='mix', type_date="%d/%m/%Y")
ta = read_files(path, sheet,ta_col_ord, 0, 0, "Site ID ", ta_dates, interger,amount, ta_bill_cols)

#Replacing values
ta.loc[ta['Rate ']==0.23, 'Rate '], ta.loc[ta['Rate ']==0.21, 'Rate '], ta.loc[ta['Rate ']=='-', 'Rate '] = '23%','21%', ''
ta.loc[ta['Percentage']==1, 'Percentage'], ta.loc[ta['Percentage']=='-', 'Percentage'] = '100%', ''

ta.head(3)

,Site ID,Site Name,Address,Site Provider,Site type,Tenant/Customer,Customer Type,Initial Rent,Current Rent,REC Uplift Charge,Current Power Charge,Renegotiations,Lease Start Date,Lease End date,Right to Renew,End of Renewal Period,Payment Terms,Payment Frequency,Rent Review,Indexation Driver,Fixed Increase,CPI,Percentage,Increase frequency,Increase Date,VAT,Rate,Termination date
0,CE007,Lack West,Lack West Kilmihil Clare,Vodafone,GBT,Three Ireland Services (Hutchinston) Ltd,MNO,,14417.338624,,Vfi supply,,06/02/2006,05/02/2011,No,,Quarterly in Advance,4,CPI Applied from 01 October annually,Annually Tenant Fees,CPI/4% (the greater of),-,100%,12,01/10/2019,Yes,23%,
1,CE007,Lack West,Lack West Kilmihil Clare,Vodafone,GBT,Tetra Ireland Communications Ltd,OTMO,9500,10142.700000,,531,,01/10/2011,30/09/2021,No,,Half Yearly,2,Annually,Annually Tenant Fees,CPI,-,100%,12,01/10/2019,Yes,23%,
2,CE007,Lack West,Lack West Kilmihil Clare,Vodafone,GBT,Imagine,OTMO,12000,12000.000000,,Vfi supply,,25/07/2019,24/07/2024,No,,Quarterly In Advance,4,-,-,-,-,,0,,Yes,23%,


In [ ]:
ta.to_csv('/content/TA_Input_Ireland_20210831.csv', index=False)

check for blank fields in Lease Start Date and Lease End date, and if Lease Start Dates always be prior to end dates

In [ ]:
import datetime as dt

#Create dates in datetime format
ta['end'] = [pd.to_datetime(i, format='%d/%m/%Y') if i!= '' else datetime(1999, 1, 1, 0, 0, 0) for i in ta['Lease End date ']]
ta['start'] = [pd.to_datetime(i, format='%d/%m/%Y') if i!= '' else datetime(1999, 1, 1, 0, 0, 0) for i in ta['Lease Start Date ']]
#Create diff columns to get negative values
ta['dt_dff'] = (ta['end'] - ta['start']).dt.days

start_or_end_blank = (ta['Lease Start Date ']=='')|(ta['Lease End date ']=='')
negative_dates = (ta['dt_dff'] <= 0)
cols = ['Site ID ',"Tenant/Customer ", 'Lease Start Date ', 'Lease End date ']
ta_dates_errors = ta[start_or_end_blank|negative_dates][cols]

ta_dates_errors


,Site ID,Tenant/Customer,Lease Start Date,Lease End date


Check customer types (MNO, OTMO)

In [ ]:
df_class_error = ta[(ta['Customer Type ']=='')|~(ta['Customer Type '].isin(['MNO', 'OTMO']))][['Site ID ',"Tenant/Customer ", 'Customer Type ']]
df_class_error

,Site ID,Tenant/Customer,Customer Type


Check Tenant/Customer for blank fields

In [ ]:
df_customer_error = ta[ta["Tenant/Customer "].isin(['', None, ' '])][['Site ID ',"Tenant/Customer "]]
df_customer_error

,Site ID,Tenant/Customer


Ta Log

In [ ]:
def general_log_erros(df_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')  
    for lista in df_list:
        try:
            if not lista[1].empty:
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
        except AttributeError:
            pass
        except:
            if lista[1] is not None and lista[1].is_valid():
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
    try: 
        writer.save() 
    except IndexError:
        print('No one log to write!')

ta_df = [['Dates Errors', ta_dates_errors],
    ['Customer Type Erros', df_class_error],
    ['Tenant Errors', df_customer_error]]
path_log_ta = '/content/TA_IE_Log.xlsx'
general_log_erros(ta_df, path_log_ta)

LC Validations

In [ ]:
#Read TA Input File
def read_files(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int,\
               cols_amount, bill_cols, format='mix', type_date="%d/%m/%Y"):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df
    def date_parser(df, columns,format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)
    #header=0, names = col_name,
    df = pd.read_excel(path, sheet_name = sheetname, engine='openpyxl', skiprows = n_skiprows)
    df = df.iloc[:,n_skip_columns:]
    df = df[col_name] 
    df = df.dropna(subset=[site_index], axis=0)
    df = replace_values(df, cols_amount, 0)
    for col in cols_amount:
        df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int).apply(lambda x: f'{x:,}')
    df = replace_values(df, cols_int, 0)
    for col in cols_int:
        df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int)
    df = df.fillna('')
    df = replace_values(df, bill_cols)
    date_parser(df, cols_date, format, type_date)
    # define the entiry columns which doesn't have NaN 
    return df
lc_ord = ["Site ID ","Site Name ","Address ","Site Provider","Contract Type","Vendor"," Current Rent  ",\
          " Sublet/Uplift "," Total Rent ","Misc Fees ","Sublet Clause ","Conditions ","Uplift for Sublet ",\
          "Lease Start Date ","Lease End date ","Right to Renew ","End of Renewal Period ","Payment Terms ",\
          "Payment Frequency","Rent Review ","Fixed Increase ","Indexation Driver","CPI/Increase Type",\
          "Percentage","Increase Frequency","Increase date","VAT Subject","VAT","Termination date ","Observations"]
interger = []#'Increase frequency'
amount = []#" Current Rent  "," Total Rent "
ta_bill_cols = ["Site ID ",\
                "Lease Start Date ","Lease End date "]
dates = ["Lease Start Date ","Lease End date ", "Increase date"]             
path='/content/TowerDB_Ireland_20210630 v1.xlsx'
sheet_lc='LC_Input_Ireland_20210630'
#(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int,\
               #cols_amount, bill_cols, format='mix', type_date="%d/%m/%Y")
lc = read_files(path, sheet_lc,lc_ord, 0, 0, "Site ID ", dates, interger,amount, ta_bill_cols)
#lc["Uplift for Sublet "] = [re.sub('^€', '', str(x)) for x in lc["Uplift for Sublet "]]
#lc["Uplift for Sublet "] = lc["Uplift for Sublet "].replace('-', '')
lc.loc[lc["VAT"]==0.23, "VAT"], lc.loc[lc["Percentage"]==1, "Percentage"] = '23%', "100%"

lc.head()

In [ ]:
lc.to_csv('/content/LC_Input_Ireland_20210831.csv', index=False)

Checking Dates Fields

In [ ]:
import datetime as dt

#Create dates in datetime format
lc['end'] = [pd.to_datetime(i, format='%d/%m/%Y') if i!= '' else datetime(1999, 1, 1, 0, 0, 0) for i in lc['Lease End date ']]
lc['start'] = [pd.to_datetime(i, format='%d/%m/%Y') if i!= '' else datetime(1999, 1, 1, 0, 0, 0) for i in lc['Lease Start Date ']]
#create diff columns to get negative values
lc['dt_dff'] = (lc['end'] - lc['start']).dt.days

start_or_end_blank = (lc['Lease Start Date ']=='')|(lc['Lease End date ']=='')
negative_dates = (lc['dt_dff'] <= 0)
cols = ['Site ID ', 'Lease Start Date ', 'Lease End date ']
df_lc_dates = lc[start_or_end_blank|negative_dates][cols]

df_lc_dates

Checking amount field

In [ ]:
def check_amounts(df, df_index, columns, pattern=','):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    from collections import defaultdict

    df_am = df[columns].fillna('')
    df_am = df_am.to_dict(orient='list')

    new_dic = defaultdict(list)

    for column in set(df_am.keys()):
        for value in df_am[column]:
            if not str(value).__contains__(pattern):
                if value in (None, '', 'nan'):
                    new_dic[column].append('Blank Value')
                elif value == '0':
                    new_dic[column].append('Input 0')
                else:
                    new_dic[column].append(f'Incorret picklist value: {value}')
            else:
                new_dic[column].append('Ok')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok', np.nan)
    df_errors[df_index] = df[df_index]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()
    #df_errors = df_errors.reset_index()
    return df_errors
"""Check Amount"""
lc_col_amounts = [' Total Rent ']
df_lc_amounts = check_amounts(lc, "Site ID ", lc_col_amounts, '.')
df_lc_amounts
# No errors

,Site ID,Total Rent
0,GY025,Blank Value


Log creating

In [ ]:
def general_log_erros(df_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')  
    for lista in df_list:
        try:
            if not lista[1].empty:
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
        except AttributeError:
            pass
        except:
            if lista[1] is not None and lista[1].is_valid():
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
    try: 
        writer.save() 
    except IndexError:
        print('No one log to write!')

df_lc = [['Dates Errors',df_lc_dates_erros],
         ['Amount Errors', df_lc_amounts]]

path_lc = '/content/LC_IE_log.xlsx'
general_log_erros(df_lc, path_lc)

/usr/local/lib/python3.7/dist-packages/openpyxl/workbook/child.py:102: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
